# **Model**

In [1]:
experiment_label = 'zz_template_test'

In [2]:
user_label = 'aj'

## **Aim**

Aim for this notebook

## **Findings**

Findings for this notebook

In [3]:
#Initial imports
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.common_lib import DataReader, NBARawData
from sklearn.svm import SVC

ModuleNotFoundError: No module named 'imblearn'

## **Data input and cleansing**

In [4]:
#Load dataset using common function DataReader.read_data()
data_reader = DataReader()
# Load Raw Train Data
df_train = data_reader.read_data(NBARawData.TRAIN)
# Load Test Raw Data
df_test = data_reader.read_data(NBARawData.TEST)

NameError: name 'DataReader' is not defined

In [ ]:
#For train dataframe, remove redundant column 'Id_old'
### Removing done in Data Split function
# cols_drop = ["Id", "Id_old"]
# df_train.drop(cols_drop, axis=1, inplace=True)

### Column Strip is already done in Data Reader for Train and Test set, so comment out
# df_train.columns = df_train.columns.str.strip()
df_train.describe

In [ ]:
df_train.info()

In [ ]:
df.test.info()

In [ ]:
#For test dataframe, remove redundant column 'Id_old'
# df_test.drop(cols_drop, axis=1, inplace=True)
# df_test.columns = df_test.columns.str.strip()
# df_test.describe

## **Negative values in dataset**

In [ ]:
print(df_train.where(df_train < 0).count())

In [ ]:
# Negative values do not make sense in this context
#Define negative cleaning function
def clean_negatives(strategy, df):
    
    if strategy=='abs':
        df = abs(df)
    if strategy=='null':
        df[df < 0] = None
    if strategy=='mean':
        df[df < 0] = None
        df.fillna(df.mean(), inplace=True)      
    
    return(df)

In [ ]:
#Clean negative numbers
negatives_strategy = 'abs'

df_train = clean_negatives(negatives_strategy, df_train)
df_test = clean_negatives(negatives_strategy, df_test)

## **Feature Correlation and Selection**

In [ ]:
#Use Pearson Correlation to determine feature correlation 
pearsoncorr = df_train.corr('pearson')

#Create heatmap of pearson correlation factors
fig, ax = plt.subplots(figsize=(10,10))
sb.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.2)

In [ ]:
#Drop correlated features w/ score over 0.9 - retain "MINS", "3P MADE","FTM","REB"
#selected_features = data_reader.select_feature_by_correlation(df_train)

##  **Standard Scaling** (Optional)

In [ ]:
#Standardise scaling of all feature values
df_train_selected  = df_train[selected_features]

#Apply scaler
df_cleaned = df_train_selected.copy()
target = df_cleaned.pop('TARGET_5Yrs')

df_train_scaled = data_reader.scale_features_by_standard_scaler(df_cleaned)
df_train_scaled

### Polynomial (Optional)


In [ ]:
X_train_poly = data_reader.polynomialize_data(X_train, 2)
X_val_poly = data_reader.polynomialize_data(X_val, 2)

### Data Splitting - Train

In [ ]:
# Split the training dataset using common function data_reader.splitdata
X_train, X_val, y_train, y_val  = data_reader.split_data(df_train)
#X_train, X_val, y_train, y_val  = data_reader.split_data(df_train_scaled)

## Simple initial model

## Define pipeline

In [ ]:
# class imbalance pipeline

!pip install imblearn

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
def drop_cols

In [ ]:
def model_pipeline(algorithm):
    
    model = Pipeline([
        ('over', SMOTE(sampling_strategy=0.5)),
        ('under', RandomUnderSampler(sampling_strategy=0.75)),
        ('model', algorithm)
    ])
    return model

## Hyperparameter search

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

In [ ]:

def objective(space):
    from sklearn.model_selection import cross_val_score
    
    #define algorithm
    
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        gamma = space['gamma'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree'],
        #scale_pos_weight = space['scale_pos_weight'],
        use_label_encoder=False,
        seed=42
    )
    
    # CV of pipeline including that algorithm 
    acc = cross_val_score(model_pipeline(xgboost), X_train, y_train, cv=5, scoring="roc_auc").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

In [ ]:
#define search space

space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'gamma' : hp.quniform('gamma', 0, 0.1, 0.01),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05)
    #'scale_pos_weight' : hp.quniform('scale_pos_weight', 1, class_weight, class_weight / 5)
}

In [ ]:
# Find optimal parameters 

best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=5
)

In [ ]:
print("Best: ", best)

## Fit best model

In [ ]:
# Fit model using the best hyperparamters from the hyperparameter search.

xgboost_2 = xgb.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    gamma = best['gamma'],    
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree'],
    #scale_pos_weight = best['scale_pos_weight'],
    use_label_encoder=False,
    seed=42
)

best_model = model_pipeline(xgboost_2)

best_model.fit(X_train, y_train)

## **Model Selection and Training**

In [ ]:
#Use GridSearchCV to optimise parameters
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3, scoring="roc_auc", n_jobs=-2) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

In [ ]:
#Print the optimised parameters
print(grid.best_params_) 

In [ ]:
#Create model with the optimised parameters
model = SVC(C=100, break_ties=False, class_weight='balanced', coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma=0.0001, kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
#Store model in /models
from joblib import dump
dump(model, '../models/' + experiment_label + '.joblib')

## **Model Evaluation**

In [ ]:
#Create predictions for train and validation
y_train_preds = model.predict(X_train)
y_val_preds = model.predict(X_val)

In [ ]:
#Evaluate train predictions
#from src.models.aj_metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import classification_report
sys.path.append(os.path.abspath('..'))
from src.models.aj_metrics import confusion_matrix

In [ ]:
y_train_preds

In [ ]:
#Training performance results
print("ROC AUC Score:")
print(roc_auc_score(y_train,y_train_preds))
print(classification_report(y_train, y_train_preds)) 

In [ ]:
#Confusion matrix
print(confusion_matrix(y_train, y_train_preds))

In [ ]:
#ROC Curve
plot_roc_curve(model,X_train, y_train)

In [ ]:
#Precision Recall Curve
plot_precision_recall_curve(model,X_train,y_train)

In [ ]:
#Validation performance analysis
print("ROC AUC Score:")
print(roc_auc_score(y_val,y_val_preds))
print("Confusion Matrix:")
print(classification_report(y_val, y_val_preds)) 

In [ ]:
#Confusion matrix
print(confusion_matrix(y_train, y_train_preds))

In [ ]:
#ROC Curve
plot_roc_curve(model,X_val, y_val)

In [ ]:
#Precision Recall Curve
plot_precision_recall_curve(model,X_train,y_train)

## **Test output**

In [ ]:
#Output predictions
X_test = df_test
y_test_preds = model.predict_proba(X_test)[:,1]

In [ ]:
y_test_preds

In [ ]:
output = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p for p in y_test_preds]})

In [ ]:
output.to_csv("../reports/" + user_label + "_" + experiment_label + ".csv", index=False)

## **Outcome**

After outputting the predictions into kaggle, the final score was 0.00